In [1]:
import io
import  os
import  tensorflow as tf
import  numpy as np
from    tensorflow import keras
from utils import LoadFishDataUtil
from    tensorflow.keras import datasets, layers, optimizers, models
from    tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
import pathlib
import h5py
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
import random
print(tf.__version__)

2.0.0


In [2]:
#data_dir ='/media/xingbo/Storage/fish_identification/data/SESSION_TENT/SESSION1'
data_dir ='/media/xingbo/Storage/fish_identification/data/SESSION_AQUARIUM/SESSION_MERGE/SESSION1'
data_dir_path = pathlib.Path(data_dir)
image_count = len(list(data_dir_path.glob('*/*.png')))
print('total images:',image_count)
BATCH_SIZE = 8
IMG_SIZE=160
IMG_WIDTH=28
IMG_HEIGHT=28

total images: 3327


In [9]:
CLASS_NAMES=None
SPLIT_WEIGHTS=(0.7, 0.3, 0.0)# train cv val vs test
myloadData = LoadFishDataUtil(data_dir,BATCH_SIZE,IMG_WIDTH,IMG_HEIGHT,CLASS_NAMES,SPLIT_WEIGHTS)
train_dataset,val_dataset,test_dataset,STEPS_PER_EPOCH, CLASS_NAMES,class_num = myloadData.loadFishDataWithname()
input_shape=(IMG_WIDTH,IMG_HEIGHT, 1)
print('total class:',class_num)


total class: 328


In [4]:
train_dataset_num_elements = tf.data.experimental.cardinality(train_dataset).numpy()
val_dataset_num_elements = tf.data.experimental.cardinality(val_dataset).numpy()
test_dataset_num_elements = tf.data.experimental.cardinality(test_dataset).numpy()
print(f"train_dataset_num_elements {train_dataset_num_elements}, val_dataset_num_elements {val_dataset_num_elements}, test_dataset_num_elements {test_dataset_num_elements}")


train_dataset_num_elements 2328, val_dataset_num_elements 998, test_dataset_num_elements 0


In [10]:
weight_decay = 0.000

model = models.Sequential()

model.add(layers.Conv2D(64, (3, 3), padding='same', input_shape=input_shape,
                        kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))

model.add(layers.Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())

model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())

model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())

model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())

model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())

#model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.5))

model.add(layers.Flatten())
model.add(layers.Dense(1024, kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())

model.add(layers.Dense(256, kernel_regularizer=regularizers.l2(weight_decay)))

#model.add(layers.Activation('softmax'))

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=None), # No activation on final dense layer
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings

])

In [11]:
def _normalize_img(img, label):
    img = tf.cast(img, tf.float32) / 255.
    #print(label)
    return (img, label)

train_dataset, test_dataset = tfds.load(name="mnist", split=['train', 'test'], as_supervised=True)
train_dataset
# Build your input pipelines
train_dataset = train_dataset.shuffle(1024).batch(32)
train_dataset = train_dataset.map(_normalize_img)

test_dataset = test_dataset.batch(32)
test_dataset = test_dataset.map(_normalize_img)

In [ ]:
train_dataset

In [16]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

base_path= '/media/xingbo/Storage/fish_identification/data/SESSION_AQUARIUM/SESSION_MERGE/SESSION1/'
IMG_TYPES = ['.png', '.jpg']
def load_data(path=data_dir):
    directories = os.listdir(path)
    #print(directories)
    img = {}
    for directory in directories:
        cur_path = os.path.join(base_path, directory)
        #print(cur_path)
        _, _, filenames = next(os.walk(cur_path))
        for filename in filenames:
            if os.path.splitext(filename)[1] in IMG_TYPES:
                if directory not in img:
                    img[directory ] = [os.path.join(base_path + directory , filename)]
                else:
                    img[directory].append(
                        os.path.join(base_path + directory , filename))

    label_encode = le.fit_transform(list(img.keys()))
    return img, label_encode

In [18]:
from itertools import islice
img, label_encode = load_data(data_dir)
inc = iter(img.items())
splitting_point = int(len(label_encode)*0.8)
x_train = dict(islice(inc, splitting_point))
x_test = dict(inc)
y_train = label_encode[:splitting_point]
y_test = label_encode[splitting_point:]
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))


ValueError: Dimensions 11 and 12 are not compatible

In [ ]:
from itertools import islice
img, label_encode = load_data(data_dir)

def tfdata_generator(images, labels, is_training, batch_size=128):
    '''Construct a data generator using tf.Dataset'''
    num_labels = len(images)
    def preprocess_fn(filename, label):
        '''A transformation function to preprocess raw data
        into trainable input. '''
        image_string = tf.io.read_file(filename)

        #Don't use tf.image.decode_image, or the output shape will be undefined
        image = tf.image.decode_png(image_string, channels=3)

        #This will convert to float values in [0, 1]
        image = tf.image.convert_image_dtype(image, tf.float32)

        resized_image = tf.image.resize(image, [IMG_WIDTH, IMG_HEIGHT])
        return resized_image, label


    def generator():
        while True:
            # Sample the labels that will compose the batch
            chosen_labels = random.sample(range(num_labels),
                                        num_classes_per_batch)
            for label in chosen_labels:
                for _ in range(num_images_per_class):
                    # yield images[label][np.random.choice(range(num_images_per_class))]
                    yield label_encode[label]

    datasets = []
    for i in range(len(img)):
        filenames = img[le.inverse_transform([i])[0]]
        labels = [i for _ in range(len(filenames))]
        dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
        datasets.append(dataset)
    
    # dataset = [tf.data.Dataset.from_tensor_slices((images, label_encode))]

    choice_dataset = tf.data.Dataset.from_generator(generator, tf.int64)
    # for d in choice_dataset:
    #     print(d)
    dataset = tf.data.experimental.choose_from_datasets(datasets, choice_dataset)
    # for image_number, label in dataset.batch(4).take(20):
    #     print("images {} from classes {}".format(image_number.numpy(), label.numpy()))
    dataset = dataset.map(preprocess_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size, drop_remainder=True if is_training else False)

    dataset = dataset.repeat()
    dataset = dataset.prefetch(1)
    return dataset


inc = iter(img.items())
splitting_point = int(len(label_encode)*0.8)
x_train = dict(islice(inc, splitting_point))
x_test = dict(inc)
y_train = label_encode[:splitting_point]
y_test = label_encode[splitting_point:]
train_dataset = tfdata_generator(x_train, y_train, is_training=True)
val_dataset = tfdata_generator(x_test, y_test, is_training=True)


In [ ]:

    def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
      img = tf.image.decode_jpeg(img, channels=3)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
      img = tf.image.convert_image_dtype(img, tf.float32)
    #img = (img/127.5) - 1
    # resize the image to the desired size.
      return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])
    
    def process_path_withname(file_path):
      label = get_label_withname(file_path)
    # load the raw data from the file as a string
      img = tf.io.read_file(file_path)
      img = decode_img(img)
      #label=label.astype('int64')
      return img, label
    
    def get_label_withname(file_path):
    # convert the path to a list of path components
      parts = tf.strings.split(file_path, '/')
      #file_path=tf.dtypes.cast(file_path, tf.int64)
      #parts= file_path.split(os.sep)

    # The second to last is the class-directory
      
      wh = tf.where(tf.equal(CLASS_NAMES,parts[-2]))
      return wh
  

In [ ]:
list_ds = tf.data.Dataset.list_files(str(data_dir+'/*/*'))
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
labeled_ds = list_ds.map(process_path_withname, num_parallel_calls=tf.data.experimental.AUTOTUNE)
   
train_size = int(0.7* image_count)
val_size = int(0.3 * image_count)
ds = labeled_ds.cache()
ds = ds.batch(BATCH_SIZE)
ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
ds = ds.shuffle(buffer_size=1000,reshuffle_each_iteration = False )
        # Build your input pipelines
train_dataset = ds.take(train_size)
test_dataset = ds.skip(train_size)



In [12]:
# build model and optimizer
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])
model.compile(optimizer=keras.optimizers.Adam(0.001),
              loss=tfa.losses.TripletSemiHardLoss()
              #loss=keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy']
             )
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 28, 28, 64)        640       
_________________________________________________________________
activation_14 (Activation)   (None, 28, 28, 64)        0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 28, 28, 64)        256       
_________________________________________________________________
dropout_9 (Dropout)          (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 28, 28, 64)        36928     
_________________________________________________________________
activation_15 (Activation)   (None, 28, 28, 64)        0         
_________________________________________________________________
batch_normalization_15 (Batc (None, 28, 28, 64)       

In [ ]:
#from tensorflow.keras.utils import plot_model
#plot_model(model, to_file='model.png')

In [ ]:
n=0
inc = iter(train_dataset)
while True:
    try:
        image_batch, label_batch = next(inc)
        imgs = image_batch.numpy()
        labels = label_batch.numpy()
        for i in range(len(imgs)):
            #print(imgs[i])
            print(labels[i])
            n = n + 1
            #print(n)
    except:
        break
   

print(n)


## training 


In [15]:
epochs = 50
validation_steps = 20
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
# This callback will stop the training when there is no improvement in
# the validation loss for three consecutive epochs.
# train
history=model.fit(train_dataset, 
                  epochs=epochs
                  #steps_per_epoch=100
                  #callbacks=[callback],
          #validation_data=val_dataset,verbose=1
                  )


Epoch 1/50
1875/1875 [==============================] - 28s 15ms/step - loss: 2.6365
Epoch 2/50
1875/1875 [==============================] - 28s 15ms/step - loss: 4.2109
Epoch 3/50
1875/1875 [==============================] - 28s 15ms/step - loss: 1.9637
Epoch 4/50
1875/1875 [==============================] - 28s 15ms/step - loss: 1.1281
Epoch 5/50
1349/1875 [====================>.........] - ETA: 7s - loss: 1.7303

KeyboardInterrupt: 

In [ ]:
# evaluate on test set
#scores = model.evaluate(test_dataset, verbose=1)
#print("Final test loss and accuracy :", scores)

In [ ]:
results = model.predict(test_dataset)
# Save test embeddings for visualization in projector
np.savetxt("vecs.tsv", results, delimiter='\t')

out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for img, labels in tfds.as_numpy(test_dataset):
    [out_m.write(str(x) + "\n") for x in labels]
out_m.close()


In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## Save model

In [ ]:
# Save the model
# model.save('vggfish.h5py')

In [ ]:
model.save_weights('model/vggfish20191125AQU_MERGE.h5')

In [ ]:
model.load_weights('model/vggfish20191125AQU_MERGE.h5')

# extract feature
## SESSION3

In [ ]:
test_data_dir ='/media/xingbo/Storage/fish_identification/data/SESSION_AQUARIUM/SESSION3'
BATCH_SIZE = 8
testloadData = LoadFishDataUtil(test_data_dir,BATCH_SIZE,IMG_WIDTH,IMG_HEIGHT,CLASS_NAMES)
sess2_test_dataset,sess2_class_num = testloadData.loadTestFishData()
test_num_elements = tf.data.experimental.cardinality(sess2_test_dataset).numpy()
print(f"we have total {test_num_elements} batches of images for testing, around {test_num_elements*BATCH_SIZE} samples")
# evaluate on test set
#scores = model.evaluate(sess2_test_dataset, verbose=1)
#print("Final test loss and accuracy :", scores)
from numpy import linalg as LA
feats = []
names = []
feature_model = models.Model(inputs=model.input, outputs=model.get_layer('batch_normalization_13').output)
n = 0
for image_batch, label_batch in sess2_test_dataset:
    feature=model(image_batch)
    
    #print(n)
    #print(feature.shape[0])
    for i in range(feature.shape[0]):
        n=n+1
        feats.append(feature[i])
        names.append(np.argwhere(label_batch[i]).ravel())
        indxmax=np.argmax(feature[i])
        #print('predictions max index:',indxmax)
        #print('predictions:', CLASS_NAMES[indxmax] )
        #print('real:', CLASS_NAMES[np.argwhere(label_batch[i]).ravel()])
       
print(f"finanly we have {n} samples extracted features")
feats3 = np.array(feats)
names3 = np.array(names)


In [ ]:
test_data_dir ='/media/xingbo/Storage/fish_identification/data/SESSION_AQUARIUM/SESSION2'
BATCH_SIZE = 8
testloadData = LoadFishDataUtil(test_data_dir,BATCH_SIZE,IMG_WIDTH,IMG_HEIGHT,CLASS_NAMES)
sess2_test_dataset,sess2_class_num = testloadData.loadTestFishData()
test_num_elements = tf.data.experimental.cardinality(sess2_test_dataset).numpy()
print(f"we have total {test_num_elements} batches of images for testing, around {test_num_elements*BATCH_SIZE} samples")

feats = []
names = []
feature_model = models.Model(inputs=model.input, outputs=model.get_layer('batch_normalization_13').output)
n = 0
for image_batch, label_batch in sess2_test_dataset:
    feature=model(image_batch)
    for i in range(feature.shape[0]):
        n=n+1
        feats.append(feature[i])
        names.append(np.argwhere(label_batch[i]).ravel())
        indxmax=np.argmax(feature[i])
       
print(f"finanly we have {n} samples extracted features")
feats2 = np.array(feats)
names2 = np.array(names)


In [ ]:
test_data_dir ='/media/xingbo/Storage/fish_identification/data/SESSION_AQUARIUM/SESSION1_LT'
BATCH_SIZE = 8
testloadData = LoadFishDataUtil(test_data_dir,BATCH_SIZE,IMG_WIDTH,IMG_HEIGHT,CLASS_NAMES)
sess2_test_dataset,sess2_class_num = testloadData.loadTestFishData()
test_num_elements = tf.data.experimental.cardinality(sess2_test_dataset).numpy()
print(f"we have total {test_num_elements} batches of images for testing, around {test_num_elements*BATCH_SIZE} samples")

feats = []
names = []
feature_model = models.Model(inputs=model.input, outputs=model.get_layer('batch_normalization_13').output)
n = 0
for image_batch, label_batch in sess2_test_dataset:
    feature=model(image_batch)
    for i in range(feature.shape[0]):
        n=n+1
        feats.append(feature[i])
        names.append(np.argwhere(label_batch[i]).ravel())
        indxmax=np.argmax(feature[i])
       
print(f"finanly we have {n} samples extracted features")
feats1 = np.array(feats)
names1 = np.array(names)





In [ ]:
# asciiList = [n.encode("ascii", "ignore") for n in names]
# directory for storing extracted features
feats_save_path = '/media/xingbo/Storage/fish_identification/data/merge3session.h5'
print ("--------------------------------------------------")
print ("      writing feature extraction results ...")
print ("--------------------------------------------------")

h5f = h5py.File(feats_save_path, 'w')
h5f.create_dataset('Session3_features_merge3session', data=feats3)
h5f.create_dataset('Session3_names_merge3session', data=names3)

h5f.create_dataset('Session2_features_merge3session', data=feats2)
h5f.create_dataset('Session2_names_merge3session', data=names2)

h5f.create_dataset('Session1_features_merge3session', data=feats1)
h5f.create_dataset('Session1_names_merge3session', data=names1)

np.save('/media/xingbo/Storage/fish_identification/data/CLASS_NAMES.npy', CLASS_NAMES)

h5f.close()

# load features

In [ ]:

feats_save_path = '/media/xingbo/Storage/fish_identification/data/merge3session.h5'
print ("--------------------------------------------------")
print ("      reading feature extraction results ...")
print ("--------------------------------------------------")

h5f = h5py.File(feats_save_path, 'r')
Session1_features = h5f['Session1_features_merge3session'][:]
Session1_names = h5f['Session1_names_merge3session'][:]

Session2_features = h5f['Session2_features_merge3session'][:]
Session2_names = h5f['Session2_names_merge3session'][:]

Session3_features = h5f['Session3_features_merge3session'][:]
Session3_names = h5f['Session3_names_merge3session'][:]

h5f.close()
        

# matching session1 vs session 2

In [ ]:
# extract query image's feature, compute simlarity score and sort
scores = np.dot(Session1_features, Session2_features.T)
print(scores.shape)
from numpy.linalg import norm
res = 1 - np.dot(Session1_features/norm(Session1_features, axis=1)[...,None],(Session2_features/norm(Session2_features,axis=1)[...,None]).T)
res = 1-res/2
lable = Session1_names == Session2_names.T
lable2 = Session1_names != Session2_names.T

gscores=res[lable]
print(gscores.shape)
iscores=res[lable2]
print(iscores.shape)

In [ ]:
import bob.measure
iscores1vs2 = iscores.astype('float64')
gscores1vs2=gscores.astype('float64') 

In [ ]:
EER = bob.measure.eer(iscores1vs2, gscores1vs2)
print(f"we can achieve EER with {EER}")

In [ ]:
from matplotlib import pyplot
# we assume you have your negatives and positives already split
npoints = 100
bob.measure.plot.roc(iscores1vs2, gscores1vs2, npoints, color=(0,0,0), linestyle='-', label='test') 
pyplot.xlabel('FPR (%)') 
pyplot.ylabel('FNR (%)') 
pyplot.grid(True)
pyplot.show() 

# matching session1 vs session 3


In [ ]:
# extract query image's feature, compute simlarity score and sort
scores = np.dot(Session1_features, Session3_features.T)
print(scores.shape)
from numpy.linalg import norm
res = 1 - np.dot(Session1_features/norm(Session1_features, axis=1)[...,None],(Session3_features/norm(Session3_features,axis=1)[...,None]).T)
res = 1-res/2
lable = Session1_names == Session3_names.T
lable2 = Session1_names != Session3_names.T

gscores=res[lable]
print(gscores.shape)
iscores=res[lable2]
print(iscores.shape)

import bob.measure
iscores1vs3 = iscores.astype('float64')
gscores1vs3=gscores.astype('float64') 
EER = bob.measure.eer(iscores1vs3, gscores1vs3)
print(f"we can achieve EER with {EER}")
from matplotlib import pyplot
# we assume you have your negatives and positives already split
npoints = 100
bob.measure.plot.roc(iscores1vs3, gscores1vs3, npoints, color=(0,0,0), linestyle='-', label='test') 
pyplot.xlabel('FPR (%)') 
pyplot.ylabel('FNR (%)') 
pyplot.grid(True)
pyplot.show() 

# matching session2 vs session 3


In [ ]:
# extract query image's feature, compute simlarity score and sort
scores = np.dot(Session2_features, Session3_features.T)
print(scores.shape)
from numpy.linalg import norm
res = 1 - np.dot(Session2_features/norm(Session2_features, axis=1)[...,None],(Session3_features/norm(Session3_features,axis=1)[...,None]).T)
res = 1-res/2
lable = Session2_names == Session3_names.T
lable2 = Session2_names != Session3_names.T

gscores=res[lable]
print(gscores.shape)
iscores=res[lable2]
print(iscores.shape)

import bob.measure
iscores2vs3 = iscores.astype('float64')
gscores2vs3=gscores.astype('float64') 
EER = bob.measure.eer(iscores2vs3, gscores2vs3)
print(f"we can achieve EER with {EER}")
from matplotlib import pyplot
# we assume you have your negatives and positives already split
npoints = 100
bob.measure.plot.roc(iscores2vs3, gscores2vs3, npoints, color=(0,0,0), linestyle='-', label='test') 
pyplot.xlabel('FPR (%)') 
pyplot.ylabel('FNR (%)') 
pyplot.grid(True)
pyplot.show() 

In [ ]:
from matplotlib import pyplot
# we assume you have your negatives and positives already split
npoints = 100
bob.measure.plot.roc(iscores1vs2, gscores1vs2, npoints, color=(0,0,0), linestyle='-', label='1 vs 2') 
bob.measure.plot.roc(iscores1vs3, gscores1vs3, npoints, color=(0,0,0), linestyle='-.', label='1 vs 3') 
bob.measure.plot.roc(iscores2vs3, gscores2vs3, npoints, color=(0,0,0), linestyle=':', label='2 vs 3') 
pyplot.xlabel('FPR (%)') 
pyplot.ylabel('FNR (%)') 
pyplot.legend()
pyplot.grid(True)
pyplot.show() 


# intra session

In [ ]:
import bob.measure
from matplotlib import pyplot
from numpy.linalg import norm



res = 1 - np.dot(Session1_features/norm(Session1_features, axis=1)[...,None],(Session1_features/norm(Session1_features,axis=1)[...,None]).T)
res = 1-res/2
lable = Session1_names == Session1_names.T
lable2 = Session1_names != Session1_names.T

gscores=res[lable]
iscores=res[lable2]

iscores1vs1 = iscores.astype('float64')
gscores1vs1=gscores.astype('float64') 
EER1vs1 = bob.measure.eer(iscores1vs1, gscores1vs1)
print(f"we can achieve EER with {EER1vs1}")

res = 1 - np.dot(Session2_features/norm(Session2_features, axis=1)[...,None],(Session2_features/norm(Session2_features,axis=1)[...,None]).T)
res = 1-res/2
lable = Session2_names == Session2_names.T
lable2 = Session2_names != Session2_names.T

gscores=res[lable]
iscores=res[lable2]

iscores2vs2 = iscores.astype('float64')
gscores2vs2=gscores.astype('float64') 
EER2vs2 = bob.measure.eer(iscores2vs2, gscores2vs2)
print(f"we can achieve EER with {EER2vs2}")

res = 1 - np.dot(Session3_features/norm(Session3_features, axis=1)[...,None],(Session3_features/norm(Session3_features,axis=1)[...,None]).T)
res = 1-res/2
lable = Session3_names == Session3_names.T
lable2 = Session3_names != Session3_names.T

gscores=res[lable]
iscores=res[lable2]

iscores3vs3 = iscores.astype('float64')
gscores3vs3=gscores.astype('float64') 
EER3vs3 = bob.measure.eer(iscores3vs3, gscores3vs3)
print(f"we can achieve EER with {EER3vs3}")


from matplotlib import pyplot
# we assume you have your negatives and positives already split
npoints = 100
bob.measure.plot.roc(iscores1vs1, gscores1vs1, npoints, color=(0,0,0), linestyle='-', label='inta-session1') 
bob.measure.plot.roc(iscores2vs2, gscores2vs2, npoints, color=(0,0,0), linestyle='-.', label='inta-session2') 
bob.measure.plot.roc(iscores3vs3, gscores3vs3, npoints, color=(0,0,0), linestyle=':', label='inta-session3') 
pyplot.xlabel('FPR (%)') 
pyplot.ylabel('FNR (%)') 
pyplot.legend()
pyplot.grid(True)
pyplot.show() 